# Spring Data

Things we need to connect to a database.
1. URL and PORT Number
2. Instruction Set
3. Driver
4. We need to convert response into application object

Spring Data MongoDB which is part of Spring Data project, provides simplify database access and NoSQL(MongoDB) support 

## Install Driver
In pom.xml add
```
<dependency>
    <groupId>org.mongodb</groupId>
    <artifactId>mongo-java-driver</artifactId>
    <version>{mongodb-version}</version>
</dependency>
```

or you can instead add
```
<dependency>
    <groupId>org.springframework.boot</groupId>
    <artifactId>spring-boot-starter-data-mongodb</artifactId>
</dependency>
```

Because `spring-boot-starter-data-mongodb` includes the `mongo-java-driver` as a transitive dependency, it automatically brings in the MongoDB Java driver as a dependency.

The `spring-boot-starter-data-mongodb` dependency pulls in serveral transitive dependencies, including:
- Spring Data MongoDB: Provides the core functionality for MongoDB integration.
- MongoDB Java Driver: The official MongoDB Java driver.

We did not specify the version for `spring-boot-starter-data-mongodb` because the version is managed by the Spring Boot parent POM (Project Object Model) which simplifies dependency management and ensures compatibility between different Spring Boot modules.

# Document
In MongoDB a `Document` is a datastructure that stores data in BSON(Binary JSON). A Document consists of key-value pairs, wehre the keys are string and values can be a variety of data types. 

The MongoDB JAva driver, `Document` class is part of the `org.bson` package and represents a BSON document.
```
org.bson.Document;
```
This class provides a way to create, manipulate and query BSON documents in Java application.

The `Document` class in MongoDB Java driver is built on top of a map structure. 
```
public class Document extends BasicBSONObject
    private static final long serialVersionUID = -4415279469780082174L;

    // Internal map to store key-value pairs
    private final Map<String, Object> map;

    public Document() {
        map = new HashMap<>();
    }

    public Document(String key, Object value) {
        this();
        map.put(key, value);
    }

    @Override
    public Object put(String key, Object value) {
        return map.put(key, value);
    }

    ...

}
```
Methods of the Document object
- `append(String key, Object value)`: Adds a key-value pair to th document and returns the document itself.
- `put(String key, Object value)`: Adds or updates a key-value pair.
- `get(String key)`: Retrieves the value associated with the specified key.
- `getString(String key)`: Retrieves value as a string.
- `getInteger(String key)`: Retrieves value as an integer.
- `getDouble(String key)`: Retrieves value as a double.
- `getBoolean(String key)`: Retrieves value as a boolean.
- `getDate(String key)`: Retrieves value as a data.
- `getList(String key, Class<E> clazz)`: Retrieves the value as a list of the specified class.
- `remove(Object key)`: Removes the key-value pair with the specified key.
- `toJson()`: Converts the document to a JSON string.

Both the put and append method can be use interchangeably, the only difference is the the append method allows you to chain methods.
```
Document doc = new Document();
doc.put("firstname", "Alice");

doc.append("lastname", "Williams")  // Adds a key-value pair
   .append("age", 30)        // Chains another append call
   .append("city", "Anytown"); // Chains another append call
```

# MongoClient
In order for your application to communicate with MongoDB you need to complete the following steps.
1. Get Mongo Client Object
2. Get Database
3. Get Collection
4. Fire Query
5. Get Result Set
6. Close Connection

To use MongoDB in Spring Boot application you don't need to cofigure `MongoClient` manually because Spring Data MongoDB handles it for you. But if you need to customize the `MongoClient`, you can do so by defining the `MongoClient` bean in your Spring configuration.

### Install dependency
Fist we need to install the necessary dependency, by adding the following into your `pom.xml`
```
<dependencies>
    <dependency>
        <groupId>org.springframework.boot</groupId>
        <artifactId>spring-boot-starter-data-mongodb</artifactId>
    </dependency>
</dependencies>
```

### Automatic Configuration with Spring Boot
Spring boot autoconfigures `MongoClient` by default, all you need to do is to provide mongodb uri and database in the `application.properties` file
```
spring.data.mongodb.uri=<mongodburi>
```
this satisfies step 1
1. Get Mongo Client Object

### Manual Configuration
If you want to customize the `MongoClient` configuration, you can define the `MongoClient` bean in your Spring configuration class
```
@Configuration
public class MongoConfig {

    @Bean
    public MongoClient mongoClient() {
        return MongoClients.create("<mongodburi>"); // Step 1. Get Mongo Client Object
    }

    @Bean
    public UserRepository userRepository(MongoClient mongoClient) {
        MongoDatabase mongoDatabase = mongoClient.getDatabase("<databasename>"); // Step 2. Get database
        MongoCollection<Document> usersCollection = mongoDatabase.getCollection("users"); // Step 3. Get collection
        return new UserRepository(usersCollection);
    }
}
```

### Convert Document to Users object
```
public class User {
    private String id;
    private String firstname;
    private String lastname;
    private int age;

    // Constructor
    public User(String id, String firstname, String lastname, int age) {
        this.id = id;
        this.firstname = firstname;
        this.lastname = lastname;
        this.age = age;
    }
    ...

    // Method to convert Document to User
    public static User fromDocument(Document document) {
        return new User(
            document.getObjectId("_id").toString(),
            document.getString("firstname"),
            document.getString("lastname"),
            document.getInteger("age")
        );
    }

    // Method to convert User to Document
    public Document toDocument() {
        Document document = new Document();
        if (this.id != null) {
            document.put("_id", new ObjectId(this.id));
        }
        document.put("firstname", this.firstname);
        document.put("lastname", this.lastname);
        document.put("age", this.age);
        return document;
    }
}
```

### Inject MongoCollection to Your Repository Class
Use Autowiring to Inject MongoCollection to your repository class.
```
@Repository
public class UserRepository {

    private final MongoCollection<Document> usersCollection;

    @Autowire
    public UserRepository(MongoCollection<Document> usersCollection) {
        this.usersCollection = usersCollection;
    }

    ...
```

## Firing Queries

### MongoCollection.find
The find operation has the following implementations
- `find()`: Finds all documents.
- `find(Bison filter)`: Finds document that match the filter.
- `find().sort(Bison sort)`: Sorts the found documents.
- `find().projection(Bison projection)`: Projects specific fields of thefound documents.
- `find().limit(int limit)`: Limits the number of documents returned.
- `find().skip(int skip)`: Skips a number of documents before returning the result set.

#### MongoCollection.find()
Finds all documents
```
List<User> users = new ArrayList<>();
usersCollection.find().iterator().forEachRemaining(doc -> users.add(User.fromDocument(doc)));
```

#### MongoCollection.find(Bson filter)
Finds documents using the match filter
```
List<User> users = new ArrayList<>();
Bson filter = Filters.eq("firstname", "John");
usersCollection.find(filter.iterator().forEachRemaining(doc -> users.add(User.fromDocument(doc)));
```

#### MongoCollection.find().sort(Bson sort)
Sorts the found documents
```
List<User> users = new ArrayList<>();
Bson sort = Sorts.ascending("lastname");
usersCollection.find(filter.iterator().forEachRemaining(doc -> users.add(User.fromDocument(doc)));
```

#### MongoCollection.find().projection(Bson projection)
Projects specific fields of the found documents
```
Bson projection = Projections.fields(Projections.include("firstname", "lastname"), Projections.excludeId());
List<User> users = new ArrayList<>();
usersCollection.find().projection(projection).iterator().forEachRemaining(doc -> users.add(User.fromDocument(doc)));
```
The above code only `firstname` and `lastname` are included in projection, `_id` and `age` are excluded. The reason why we are explicitly excluding the id because the id field are projected by default so we need to specify that the id field is excluded.

input
```
{
    "id": 1,
    "firstname": "grace",
    "lastname": "hopper",
    "age": 55
},
{
    "id": 2,
    "firstname": "grace",
    "age": 55
},
{
    "id": 3,
    "firstname": "grace",
    "lastname": "hopper"
}
```
output
```
{
    "firstname": "grace",
    "lastname": "hopper"
},
{
    "firstname": "grace"
},
{
    "firstname": "grace",
    "lastname": "hopper"
}
```

Explicit Exclution<br>
you can explicitly exclude the lastname field while including the other fields except for id.
```
Bson projection = Projections.fields(Projections.exclude("lastname"), Projections.excludeId());
List<User> users = new ArrayList<>();
usersCollection.find().projection(projection).iterator().forEachRemaining(doc -> users.add(User.fromDocument(doc)));
```
Input
```
{
    "id": 1,
    "firstname": "john",
    "lastname": "mak",
    "age": 55
},
{
    "id": 2,
    "firstname": "john",
    "age": 55
},
{
    "id": 3,
    "firstname": "john",
    "lastname": "mak"
}
```

output
```
{
    "firstname": "john",
    "age": 55
},
{
    "firstname": "john",
    "age": 55
},
{
    "firstname": "john"
}
```

#### MongoCollection.find().limit(int limit)
Limits the number of documents returned
```
int limit = 10;
List<User> users = new ArrayList<>();
usersCollection.find().limit(limit).iterator().forEachRemaining(doc -> users.add(User.fromDocument(doc)));
```

#### MongoCollection.find().skip(int skip)
Skips a number of documents before returning the result set:
```
int skip = 5;
List<User> users = new ArrayList<>();
usersCollection.find().skip(skip).iterator().forEachRemaining(doc -> users.add(User.fromDocument(doc)));
```

#### Pagination
Pagination in MongoDB is implemented using the `skip` and `limit` in combination with the `find` method.
```
List<User> findAll(int pageNumber, int pageSize) {
    int skip = (pageNumber - 1) * pageSize;

    List<User> users = new ArrayList<>();
    usersCollection.find().skip(skip).limit(pageSize)
        .forEachRemaining(doc -> users.add(User.fromDocument(doc)));
    return users;
}
```

### findOne
We can use the `first` to get exactly 1 document. The below code returns a User by id.
```
String id = "60c72b2f9b1e8a5b1c4e4d4a";
Bson filter = new Document(Filters.eq("_id", new ObjectId(id)), new ObjectId(id));
User.fromDocument(usersCollection.find(filter.first());
```

### InsertOne
The `insertOne` inserts a single document into the collection and returns `InsertOneResult` which contains the `_id` of the inserted document.

`InsertOneResult`: primary provides a single piece of information, the ID of the inserted document which is accessable using `getInsertedId()`
```
User insertOne(User user) {
    InsertOneResult result = usersCollection.insertOne(user.toDocument());
    ObjectId id = result.getInsertedId().asObjectId().getValue();
    user.setId(id.toString());
    return user;
}
```

### InsertMany
The `insertMany` method inserts multiple documents into a collection.
```
List<User> insertMany(List<User> users) {
    List<Document> documents = users.stream().map(User::toDocument).toList();

    InsertManyResult result = usersCollection.insertMany(documents);

    List<User> insertedUsers = new ArrayList<>();
    int i = 0;
    for (BsonValue id : result.getInsertedIds().values()) {
        User user = users.get(i);
        user.setId(id.asObjectId().getValue().toHexString());
        insertedUsers.add(user);
        i++;
    }

    return insertedUsers;
}
```

### UpdateOne
The `updateOne` method updates a single document that matches the filter criteria.
```
User updateOne(String id, User user) {
    UpdateResult result = usersCollection.updateOne(
        Filters.eq("_id", new ObjectId(id)),
        new Document("$set", user.toDocument()));

    if (result.getMachedCount() == 1 && result.getModifiedCount() == 1) {
        user.setId(id);
        return user;
    } else {
        throw new MongoException("Failed to update document with ID: " + id);
    }
}
```

### UpdateMany
The `updateMany` method updates all documents that match the filter criteria.
```
long updateMany(Bson filter, User user) {
    return usersCollection.updateMany(
            filter,
            new Document("$set", user.toDocument()))
        .getModifiedCount();
}
```

### DeleteOne
The `deleteOne` method deletes a single document that matches the filter criteria.
```
void deleteOne(String id) {
    DeleteResult result = usersCollection.deleteOne(Filters.eq("_id", new ObjectId(id));
    if (result.getDeletedCount() != 1) {
        throw new MongoException("Failed to delete document with ID: " + id);
    }
}
```

### DeleteMany
The `deleteMany` method deletes all documents that match the filter criteria.
```
long deleteMany(Bson filter) {
    return usersCollection.deleteMany(filter).getDeletedCount();
}
```

## Close Connection
Since Spring handles lifecycle of the MongoClient bean, it will handle closing the connection when the application context is closed.

## Filter Operators
Operators used to filter documents based on specific operator
- `$in`
- `$match`
- `$eq`
- `$gt`
- `$lt`
### $in
the `$in` operator is sued to select documents where the value from a field matches any value in the specific array. It is similar o the SQL `IN` clause.
```
{
    <field>: { $in: [<value1>, <value2>, ..., <valueN>] }
}
```

Find documents where the `status` field is either "A", "D", or "P":
```
Bdon filter = Filters.in("status", Arrays.asList("A", "D", "P"));
FindIterable<Document> result = collection.find(filter);
```

Find documents where the `_id` field match list of ids:
```
List<ObjectId> ids = Arrays.asList(new ObjectId("60b8d2953b48c18a0e6399a0"), new ObjectId("60b8d2953b48c18a0e6399a1"));
Bson filter = Filters.in("_id", ids);
FindIterable<Document> result = collection.find(filter);
```
### $match
The `$match` operator is used in the aggregation pipeline to filter documents. The `$match` stage filters the documents to pass only those that match the specified conditions to the next stage of the pipeline.
```
{
    $match: { <query> }
}
```
Using `$match` to filter documents based on multiple criteria.
```
List<Bson> pipeline = Arrays.asList(
    Aggregates.match(Filters.and(
        Filters.eq("status", "A"),
        Filters.gte("age", 21)
    ))
);
AggregateIterable<Document> result = collection.aggregate(pipeline);
```

## Update Operators
Operators used to update documents
- `$set`
- `$unset`
- `$inc`
- `$desc`

### $set
The `$set` operator is used to update the value of a field in a document. It can be used in update operations such as `updateOne`, `updateMany`, or in an update stage of an aggregation pipeline.
```
{
    $set: {
        <field1>: <value1>,
        <field2>: <value2>,
        ...
    }
}
```
```
User updateOne(String id, User user) {
    UpdateResult result = usersCollection.updateOne(
        Filters.eq("_id", new ObjectId(id)),
        new Document("$set", user.toDocument()));

    if (result.getMachedCount() == 1 && result.getModifiedCount() == 1) {
        user.setId(id);
        return user;
    } else {
        throw new MongoException("Failed to update document with ID: " + id);
    }
}
```

## Accumulate Operators
Accumulator operators are used during aggregation in MongoDB, specifically in stages like `$group`, `$bucket`, `$bucketAuto`, and `$setWindowFields` to perform calculations and computation on grouped data. These operators aggregate data across multiple documents and result a computed result.
- `$sum`: Calculates the sum of numeric values. Can also be used to count the number of documents.
- `$avg`: Calculates the averge of numeric values.
- `$min`: Finds the minimum value in a group.
- `$max`: Finds the maximum value in a group.
- `$first`: Returns the first value in a group.
- `$last`: Returns the last value in a group.
- `$push`: Adds values to an array.
- `$addToSet`: Adds values to an array, but only includes unique values.

## Aggregation
Aggeration in MongoDB processess data records and returns computed result.

Aggreation operations can be performed using the `aggregate` method. Which accepts a pipeline of stages, each perofrming a different operation. Each stage transforms the documents as they pass through the pipeline.

Key Aggregation Stages
1. `$match`: Filters the documents to pass only those that match the specified conditions to the next pipeline stage.
2. `$group`: Groups document by a specific key and performed aggregation operations, such as sum, average count, etc.
3. `$sort`: Sorts the documents by a specified field in ascending or descending order.
4. `$project` Reshapes each document by including, excluding, or addign new fields.
5. `$limit` Limits the number of documents passed to the next stage.
6. `$skip`: Skips the specified number of documents.
7. `$set`: Adds new fields or updates existing fields in the documents.
8. `$unwind`: Deconstructs an array field fro mthe input documents to output a document for each element.

Ex. Here is an example of taking a user attribute as input and returns the count of documents grouped by attribute.

```
List<JsonNode> countByAttribute(String attribute) {
    // Aggreation pipeline
    List<Bson> pipeline = Arrays.asList(
        // Group by a specified attribute and count the number of documents in each group
        Aggregates.group("$" + attribute, new Document("count", new Document("$sum", 1))),
        // Project the results to include the attribute and count
        Aggregates.project(new Document("_id", 0)
                            .append(attribute, "$_id")
                            .append("count", "$count"))
    );

    // Execute the aggregation
    AggregateIterable<Document> result = usersCollection.aggregate(pipeline);

    // Convert the result to a list of JSON objects
    List<JsonNode> jsonResults = new ArrayList<>();
    for (Document doc : result) {
        try {
            JsonNode jsonNode = mapper.readTree(doc.toJson();
            jsonResults.add(jsonNode);
        } catch (Exception e) {
            e.printStrackTrace();
        }
    }

    return jsonResults;
}
```
In the above
1. The Aggregation Pipeline
- - `$group`: Groups the document by the specified attribute and counts the number of documents in each group using `$sum`.
  - `$project`: Reshapes the documents to include only the attribute and count.
2. Execution:
- - The `aggregate` method executes the aggregation pipeline on `usersCollection`.
3. Result Conversion:
- - Converts the aggregation result into a list of JSON objects uing Jackson's `ObjectMapper`.
 
If we aggregate by status
```
countByAttribute("status");
```
input
```
[
    { "_id": 1, "firstname": "John", "lastname": "Doe", "age": 25, "status": "active" },
    { "_id": 2, "firstname": "Jane", "lastname": "Doe", "age": 30, "status": "inactive" },
    { "_id": 3, "firstname": "Jim", "lastname": "Beam", "age": 25, "status": "active" },
    { "_id": 4, "firstname": "Jack", "lastname": "Daniels", "age": 35, "status": "active" },
    { "_id": 5, "firstname": "Johnny", "lastname": "Walker", "age": 30, "status": "inactive" }
]
```
output
```
{
  "status": "active",
  "count": 3
}
{
  "status": "inactive",
  "count": 2
}
```

### AggregateIterable
`AggregateIterable` is an iterable of documents. In MongoDB represents the result of an aggregate operation and provides various method to access and process the report. 

The result set contains the documents that are produced by the aggregation pipeline.

### Attribute Path
In MongoDB, an attribute path (or ffield path) is used to reference fields within documents. When working with MongoDB aggregation pipelines, you often need to specify which fields to operate on. These field referenece typically start with a `$` symbol to indicate that they are paths to fields in the documents being processed.

Why use attribute path? <br>
Using attribute paths allows MongoDB to understand which fields you are referenceing within your documents. This is often used in aggregation pipelines shere you want to group by, projects or perform operations on specific fields.

Constructing Attribute Paths
1. Basic Field Reference:
- - When you want to reference a field directly, you prepend the field name with `$`
  - eg. `$status`
2. Dynamic Field Reference:
- - If you want dynamically build the field reference, you construct the attribute path by concatenating `$` with the field name.
  - eg. `"$" + attribute`
 
Aggregate Pipeline:
- - `$group`
  - - `Aggregates.group("$" + attribute, new Document("count", new Document("count", new Document("$sum", 1)))`
    - - Reshapes the output to include the attribute name and the count.
      - The `_id` field (which holds the group key) is renamed for the attribute name for clarity.
     
## Recap
List<User> findAll()
User findById(String id)
User insertUser(User user)
 updateUser(String id, User user)
 deleteUser(String id)

1. Install dependencies
`pom.xml`
```
<dependencies>
    <dependency>
        <groupId>org.springframework.boot</groupId>
        <artifactId>spring-boot-starter-data-mongodb</artifactId>
    </dependency>
</dependencies>
```

2. Configure MongoClient
`MongoConfig.java`
```
@Configuration
public class MongoConfig {

    @Bean
    public MongoClient mongoClient() {
        return MongoClients.create("<mongodburi>"); 
    }

    @Bean
    public UserRepository userRepository(MongoClient mongoClient) {
        MongoDatabase mongoDatabase = mongoClient.getDatabase("<databasename>"); 
        MongoCollection<Document> usersCollection = mongoDatabase.getCollection("users"); 
        return new UserRepository(usersCollection);
    }
}
```

3. Define EntityObject
`User.java`
```
public class User {
    private String id;
    private String firstname;
    private String lastname;
    private int age;

    // Constructor
    public User(String id, String firstname, String lastname, int age) {
        this.id = id;
        this.firstname = firstname;
        this.lastname = lastname;
        this.age = age;
    }
    ...

    // Method to convert Document to User
    public static User fromDocument(Document document) {
        return new User(
            document.getObjectId("_id").toString(),
            document.getString("firstname"),
            document.getString("lastname"),
            document.getInteger("age")
        );
    }

    // Method to convert User to Document
    public Document toDocument() {
        Document document = new Document();
        if (this.id != null) {
            document.put("_id", new ObjectId(this.id));
        }
        document.put("firstname", this.firstname);
        document.put("lastname", this.lastname);
        document.put("age", this.age);
        return document;
    }
}
```

4. Autowire MongoCollection into your Repository
`UserRepository.java`
```
@Repository
public class UserRepository {
    private final MongoCollection<Document> usersCollection;

    @Autowired
    public UserRepository(MongoCollection<Document> usersColleciton) {
        this.usersCollection = usersCollection;
    }

    public List<User> findAll() {
        List<User> users = new ArrayList<>();
        for (Document doc: usersCollection.find()) {
            users.add(User.fromDocument(doc));
        }
        return users;
    }

    public List<User> findAll(int pageNumber, int pageSize) {
        int skip = (pageNumber - 1) * pageSize;

        List<User> users = new ArrayList<>();
        usersCollection.find().skip(skip).limit(pageSize)
            .forEachRemaining(doc -> users.add(User.fromDocument(doc)));
        return users;
    }

    public User findById(String id) {
        Document doc = usersCollection.find(Filters.eq("_id", new ObjectId(id))).first();
        return doc != null ? User.fromDocument(doc) : null;
    }

    publc User insertUser(User user) {
        Document doc = user.toDocument();
        usersCollection.insertOne(doc);
        user.setId(doc.getObjectId("_id").toHexString());
        return user;
    }

    public User updateUser(String id, User user) {
        Bson filter = Filter.eq("_id", new ObjectId(id));
        Document updateDocument = new Document($set", user.toDocument());

        UpdateResult result = usersColleciotn.updateOne(filter, updateDocument);

        if (result.getMatchedCount() > 0) {
            user.setId(id);
            return user;
        } else
            return null;
        }
    }

    public void deleteUser(String id)
        Bson filter = Filters.eq("_id", new ObjectId());
        DeleteResult result = usersCollection.deleteone(filter);
        if (result.getDeletedCount() == 0) {
            throw new RuntimeException("No user found with id: " + id);
        }
    }
}
```

# MongoTemplate/MongoOperation
`MongoTemplate` from Spring Data MongoDB, provides a higher-level abstraction for MongoDB operations.

MongoTemplate implements MongoOperations. Both are part of org.springframework.data.mongodb.core package.

MongoOperations interface defines the collection of methods for performing operations on a MongoDB. 
- `<T> T insert(T objectToSave)`: Inserts a document into the default collection, returns the inserted object.
- `<T> T insert(T objectToSave, String collectionName)`: Insert a document into the specified collection, return the inserted object
- `<T> Collection<T> insert(Collection<? extends T> batchToSave, Class<?> engityClass)`: Inserts a batch of documents into the default collection, returns collection of objects
- `<T> Collection<T> insert(Collection<? extends T> batchToSave, String collectionName)`: Inserts a batch of documents into the specified collection, returns inserted collection of objects.
- `<T> T save(T objectToSave)`: Saves (inserts or updates) a document into the default colleciton, returns the saved object.
- `<T> T save(T objectToSave, String collectionName)`: Saves (inserts or updates) a document into the specified collection, returns the saved object.
- `<T> T findById(Object id, Class<T> extityClass)`: Finds a document by its ID in the default collection, returns found object or null if not found.
- `<T> T findById(Object id, Class<T> entityClass, String collectionNAme)`: finds a document by its ID in a specified collection, return s found object or null if not found.
- `boolean exists(Query query, Class<?> entityClass)`: Checks for the existence of document matching the query in the default collection, returns true if a matching document exist, false otherwise.
- `boolean exists(Query query, String collectionName)`: Checks for the existence of documents matching the query in the specified colleciton, return true if a matching document exists, false otherwise.
- `<T> List<T> find(Query query, Class<T> entityClass)`: Finds documents matching the query in the default collection, returns a list of matching documents.
- `<T> List<T> find(Query query, Class<T> entityClass, String collectionName)`: Finds documents matching the query in the specified collection, returns a list of matching documents.
- `<T> T findOne(Query query, Class<T> entityClass)`: Finds the first matching query in the default collection, returns the first matching document or null if none found.
- `<T> T findOne(Query query, Class<T> entityClass, String collectionName)`: Finds the first document matching the query in the specified collection, retruns first matching document or null if not found.
- `long count(Query query, Class<?> entityClass)`: Counts the number of documents matching the query in the default collection, returns the number of matching documents.
- `long count(Query query, Class<?> entityClass, String collectionName)`: Countes the number of documents matchign the query in the specified collection, return the number of matching documents.
- `DeleteResult remove(Object object)`: Removes a document from the default collection, returns `DelectResult` object with details of the deletion.
- `DeleteResult remove(Object object, String collectionName)`: Removes a document from the specified collection, returns `DeleteResult` object with details of the deletion.
- `DeleteResult remove(Query query, Class<?> entityClass)`: Removes documents matching the qeury from the default collection, returns `DeleteResult` object with details of the deletion.
- `DeleteResult remove(Query query, Class<?> entityClass, String collectionName)`: Removes documents matching the query from the specified collection, returns `DeleteResult` object with details of the deletion.
- `UpdateResult updateFirst(Query query, Update update, Class<?> entityClass)`: Update the first document matching the query in the default collection, returns `UpdateResult` object with details of the update.
- `UpdateResult updateFirst(Query query, Update update, Class<?> entityClass, String collectionName)`: Update the first document matching the query in the specified collection, returns `UpdateResult` object with details of the update.
- `UpdateResult updateMulti(Query query, Update update, Class<?> entityClasss)`: Updates all documents matching the query in the default collection, returns `UpdateResult` object with details of the update.
- `UpdateResult updateMulti(Query query, Update update, Class<?> entityClass, String collectionName)`: Updates all documents matching the query in the specified collection, returns `UpdateResult` object with details of the update.
- `UpdateResult upsert(Query query, Update update, Class<?> entityClass)`: Performs an upsert operation (update if exists, otherwise insert) in the default collection, returns `UpdateResult` object with details of the update.
- `UpdateResult upsert(Query query, Update update, Class<?> entityClass, String collectionName)`: Performs an upsert operation (update if exists, otherwise insert) in the specified collection, returns `UpdateResult` object with details of the update.
- `<O> AggregationResults<O> aggregate(Aggregation aggregation, Class<> inputType, Class<O> outputType`: Performs an aggregation operation on the default collection, returns `AggregationResults<O>` containing the results of the aggregation operation.
- `<O> AggregationResults<O> aggregate(Aggregation aggregation, String collectionName, Class<O> outputType)`: Performs an aggregation operation on the specified collection, returns `AggregationResult<O>` containing the results of the aggregation operation.
- `<O> Stream<O> aggregateStream(Aggregation aggregation, Class<?> inputType, Classs<O> outputType)`: Performs an aggregation operation on the default collection for the specified input type, returns `Stream<O>` a stream containing the results of the aggregation operation.
- `<O> Stream<O> aggregateStream(Aggregation aggregation, String collectionName, Class<O> outputType)`: Performs an aggregation operation on the specified collection, returns `Stream<O>` stream ontaining the results of the aggregation operation.
- `void ensureIndex(IndexOperations indexOps, Index index)`: Ensures that an index exists on the specified collection.

You might ask, what is IndexOperations, and where is the collection specified?
- - IndexOperation: Inteface for managing indexes on a collection. 
  - Specify the collection:
  - - Using Collection Name: `mongoOperations.indexOps("collectionName")`
    - Using Entity Class: `mongoOperations.indexOps(EntityClass.class)`
```
IndexOperations indexOps = mongoOperations.indexOps("users");
Index index = new Index().on("lastname", Sort.Direction.ASC);
indexOps.ensureIndex(index);
```

- `void dropIndex(IndexOpserations indexOps, String name)`: Drops an index by name fro mthe specified collection.
- `void dropAllIndexes(IndexOperations indexOps)`: Drops all indexes from the specified collection.
- `List<IndexInfo> getIndexInfo(IndexOperations indexOps)`: Retrieves information about the indexes on the specified collection, returns `List<IndexInfo>` containing information about the indexes on the collection.
- `void createCollection(Class<?> entityClass)`: Creates a collection for the specified entity class.
- `void createCollection(Class<?> entityClass, CollectionOptions collectionOptions)`: Creates a collection for the specified entity class with the given options.
- `void createCollection(String collectionName)`: Creates a collection with the specified name.
- `void createCollection(String collectionName, CollectionOptions collectionOptions)`: Creates a collection with the specified name and options.
- `void dropCollection(Class<?> entityClass)`: Drops the collection.

Comparison of Save and Insert
|Feature|Insert|Save|
|--|--|--|
|Inserts new documents|Yes|Yes|
|Updates existing documents|No|Yes|
|Behavior if document exists|Throws an exception(`DuplicateKeyException`)|Updates the existing document|
|Upsert operation|No|Yes|
|Use case|When you are sure the document is new|When you want up insert or update the document|

## Install dependency
Fist we need to install the necessary dependency, by adding the following into your pom.xml

<dependencies>
    <dependency>
        <groupId>org.springframework.boot</groupId>
        <artifactId>spring-boot-starter-data-mongodb</artifactId>
    </dependency>
</dependencies>

## Configuring MongoOperations
Autowire `MongoOperations` or `MongoTemplate`? 
Since there is no significant method in `MongoTemplate` not in `MongoOperation`(in interface `MongoTemplate` implements) we will work with `MongoOperations`

### Automatic Configuration with Spring Boot¶
Spring boot autoconfigures MongoOperations by default, all you need to do is to provide mongodb uri and database in the application.properties file

spring.data.mongodb.uri=<mongodburi>

### Manual Configuration
If you want to customize the MongoClient configuration, you can define the MongoClient bean in your Spring configuration class

```
@Configuration
public class MongoConfig {

    @Bean
    public MongoClient mongoClient() {
        return MongoClients.create("<mongodburi>");
    }

    @Bean
    public UserRepository userRepository(MongoClient mongoClient) {
        MongoOperations mongoOperations = new MongoTemplate(mongoClient,"<databasename>");
        return new UserRepository(mongoOperations);
    }
}
```
## Defining the Domain Object
By default, if the collection name is not specified, Spring Data MongoDB will use the class name of the Domain object in lowercase as the collection name. If you use `User` class, it will look for collection named user in the database.

You can override this default behavior by specifying the collection name explicitly either in the method call by using the `@Document` annotation on the class.

```
@Document(collection = "users")
public class User {
    @Id
    private String id;
    private String firstname;
    private String lastname;
    private int age;

    // Constructs, getters, and setters
}
```
In this case, the collection name `users` will be used instead of the default `user`.

You can also specify the collection name by passing the collection name in MongoOperation queries as a parameter.
```
mongoOperations.find(query, User.class, "users");
```

The difference be tween MongoOperation and MongoClient is that MongoOperations abstract away dealing with DBObjects(Documents) when querying MongoDB. When we are using MongoClient we need to convert the Domain object to Documents and vice versa. MongoOperation takes care of that for you so you need only need to provide the Domain object.

### Inject MongoCollection to Your Repository Class
Use Autowiring to Inject MongoOperations to your repository class.
```
@Repository
public class UserRepository {

    private final MongoOperations<Document> mongoOperations;

    @Autowire
    public UserRepository(MongoOperations<Document> mongoOperations) {
        this.mongoOperations = mongoOperations;
    }

    ...
```
## Query
The `Query` class is used to construct queries used MongoOperations to query documents. It provides a way to define the criteria, fields to be used in projection, pagination, sorting and other query options.

Basic Query
```
List<User> findAll() {
    Query query = new Query();
    return mongoOperations.find(query, User.clas);
}
```

### Pagination
To implement pagination, use the `skip` and `limit` methods of the Query class.
```
List<User> finalAll(int pageNumber, int pageSize) {
    Query query = new Query()
        .skip((pageNumber - 1) * pageSize)
        .limit(pageSize);
    return mongoOperations.find(query, User.class);
```

### Projection
To include or exclude specific fields in the result set, use the `fields` method along with `includ` or `exclude`:
```
List<User> findAll() {
    Query query = new Query();
        .include("firstname", "lastname")
        .exclude("_id");
    return mongoOperations.find(query, User.class);
}
```

## Criteria
The `Criteria` class provides static methods to build query conditions. These conditions are chained together to create complex queries.

Static Criteria Methods:
1. Criteria.where(String key): Creates a `Criteria` object for the specified key.
2. Criteria.is(Object value): Checks if the field is equal to the given value.
3. Criteria.gt(Object value): Checks if the field is greater than the given value.
4. Criteria.get(Object value): Checks if the field is greater than or equal to the given value.
5. Criteria.lt(Object value): Checks if the field is less than the given value.
6. Criteria.lte(Object value): Checks if the field is less than or equal to the given value.
7. Criteria.ne(Object value): Checks if the field is not equal to the given value.
8. Criteria.in(Object... value): Checks if the field is in the given values.
9. Criteria.nin(Object... value): Checks if the field is not in the given value.
10. Criteria.regex(String regex): Checks if the field matches the given regular expression.
11. Criteria.exists(boolean): Checks if the field exists or not
```
Criteria.where("department").exists(true)
```
The query will return employees with the `department` field.

## Update
The `Update` class is used in `MongoOperations` to update documents.<br>
Key Methods of the 'Update` class
1. `set(String key, Object value)`: Sets the value of a field.
2. `unset(String key)`: Removes a field from the document.
3. `inc(String key, int value)`: Increments the value of a field by the specified amount.
4. `mul(String key, int value)`: Multiplies the value of a field by the specified amount.
5. `rename(String oldName, String newName)`: Renames a field
6. `addToSet(String key, Object value)`: Adds a value to an arrayonly if it doesn't already exist in the array.
7. `push(String key, Object value)`: Adds a value to an array.
8. `pop(String key, Position position)`: Removes the first or last element of an array.
9. `pull(String key, Object value)`: Removes instances of a value from an array.
10. `pullAll(String key, Object[] values)`: Removes all instances of the specified values from an array.
11. `bitwise(String key, BitwiseOperator operator, int value)`: Performs a bitwise operation on a field.

## Aggregation
Aggregation operations in `MongoOperations` allows you to process and transform data records and return computed results.

Key Components of Aggregation 
1. Aggregation Pipelines: A sequence of stages each transforming the document.
2. Aggregation Stages: Different stages used in a pipeline, like `$match`, `$group`, `$sort`, `$project`, etc.

Aggregation Objects
1. Aggregation: This class helps build the aggregation pipeline.
2. AggregationOperation: Represents a single stage in the aggregation pipeline, such as `$match`, `$group`, `$project`, etc.
3. AggregationResults: Holds the results of an aggregation operation.
4. MatchOperation: A class that represents a $match stage in the aggregation pipeline.
5. GroupOperation: A class that represents a $group stage in the aggregation pipeline.
6. ProjectionOperation: A class that represents a $project stage in the aggregation pipeline.
7. SortOperation: A class that represents a $sort stage in the aggregation pipeline.
8. LimitOperation: A class that represents a $limit stage in the aggregation pipeline.
9. SkipOperation: A class that represents a $skip stage in the aggregation pipeline.

Ex. Here is an example of taking a user attribute as input and returns the count of documents grouped by attribute.
```
List<Map<String, Object>> countByAttribute(String attribute) {
    // Create a group operation to group by the specified attribute and count the number of documents
    GroupOperation groupByAttribute = Aggregation.group(attribute).count().as("count");

    // Create an aggregation pipeline with the gorup operation
    Aggregation aggregation = Aggregation.newAggregation(groupByAttribute);

    // Execute the aggregation and get the results
    AggregationResults<Map> results = mongoOperations.aggregation(aggregation. "user", Map.class);

    // Return the mapped results
    return results.getMappedResults();
}
```

## Recap

1. Install dependency<br>
in your `pom.xml`
```
<dependencies>
    <dependency>
        <groupId>org.springframework.boot</groupId>
        <artifactId>spring-boot-starter-data-mongodb</artifactId>
    </dependency>
</dependencies>
```

2. Manual configuration
```
@Configuration
public class MongoConfig {

    @Bean
    public MongoClient mongoClient() {
        return MongoClients.create("<mongodburi>"); 

    @Bean
    public UserRepository userRepository(MongoClient mongoClient) {
        MongoOperations mongoOperations = new MongoTemplate(mongoClient,"<databasename>");
        return new UserRepository(mongoOperations);
    }
}
```

3. Defining the Domain Object
```
@Document(collection = "user")
public class User {
    @Id
    private String id;
    private String firstname;
    private String lastname;
    private int age;

    // Constructs, getters, and setters
}
```

4. Create inject MongoOperations in your repository class
```
@Repository
public class UserRepository {

    private final MongoOperations mongoOperations;

    @Autowired
    public UserRepository(MongoOperations mongoOperations) {
        this.mongoOperations = mongoOperations;
    }

    public User findById(String id) {
        retrun mongoOperations.findById(id, User.class);
    }

    public List<User> findAll() {
        // Returns list of all User excluding the field id
        Query query = new Query();
        query.fields().exclude("_id");
        return mongoOperations.find(query, User.class);
    }

    public List<User> findAll(int pageNumber, int pageSize) {
        Query query = new Query
            .skip((pageNumber - 1) * pageSize)
            .limit(pageSize)
            .fields().exclude("_id");
        return mongoOperations.find(query, User.class);
    }

    public User insertUser(User user) {
        return mongoOperations.user(user);
    }

    public User updateUser(String id, User user) {
        Query query = new Query(Criterial.where("_id").is(id());
        Update update = new Update()
            .set("firstname", user.getFirstname())
            .set("lastname", user.getLastname())
            .set("age", user.getAge());

        // User findAndModify to update the document and return the updated document
        return mongoOperations.findAndModify(
            query,
            update,
            FindAndModifyOptions.options().returnNew(true),
            User.class);
    }

    public void deleteUser(String id) {
        Query query = new Query(Criteria.where("_id").is(id));
        mongoOperations.remove(query, User.class);
    }
}
```

# MongoRepository
`MongoRepository` is an interface that provides a simple and convenient way to perform CRUD operations and custom queries on MongoDB Collections. It extends the `CrudRepository` interface that provides default queries, and you can add custom queries using the `@Query` annotation.

Benefits of MongoRepository
- Abstract away implementation of queries. Queries is provided by the MongoRepository interface.

## Setup
Step 1. Add the following dependency in your `pom.xml`
```
<dependencies>
    <dependency>
        <groupId>org.springframework.boot</groupId>
        <artifactId>spring-boot-starter-data-mongodb</artifactId>
    </dependency>
</dependencies>
```

Step 2. Manual configuration
```
@Configuration
public class MongoConfig {
SimpleMongoClientDatabaseFactory

    @Bean
    public MongoClient mongoClient() {
        return MongoClients.create("<mongodburi>");
    }

    @Bean
    public MongoTemplate mongoTemplate() {
        return new MongoTemplate(mongoClient, "<databasename>");
    }
}
```

Step 3. Define class for domain object
```
@Document(collection = "user")
public class User {
    @Id
    private String id;
    private String firstname;
    private String lastname;
    private int age;

    // Constructs, getters, and setters
}
```

Step 4. Create your repository interface
```
public interface UserRepository extends MongoRepository<User, String> {
}
```

Step 5. Use the Repository in a Service
```
@Service
public class UserService {

    @Autowired
    private UserRepository userRepository;

    public User saveUser(User user) {
        return userRepository.save(user);
    }

    public void deleteUser(String id) {
        userRepository.deleteById(id);
    }

    public List<User> getAllUser() {
        return userRepository.findAll();
    }
}
```

## CrudRepository
`CrudRepository` is a Spring Data interface that provides generic CRUD operations. It interfaces across different data stores, including relational databases and NoSQL databases like MongoDB.

Key Methods in CrudRepository
- `save(S entity)`: Saves a given entity. If the entity is new, it is inserted otherwise it is updated
- `saveAll(Iterable<S> entities)`: Saves all given entities.
- `findById(ID id)`: Retrieves an entity by its ID.
- `existsById(ID id)`: Checks if an entity with the given ID exists.
- `findAll()`: Retrieves all entities.
- `findAllById(Iterable<ID> ids)`: Retrieves all entities by their IDs.
- `count()`: Counts the number of entities.
- `deleteById(ID id)`: Deltes the entity with the given ID.
- `delete(T entity)`: Deletes a given entity.
- `deleteAll(Iterable<? extends T> entities)`: Deletes all given entities.
- `deleteAll()`: Deletes all entities.

Features in MongoRespository not in Crud Repository.
1. `@Query` annotation for custom MongoDB queries. This is useful for complex queries
```
@Query("{ 'department' : ?0")
List<Employee> findEmployeesInDepartment(String department);
```
2. GeoSpatialQueires: Support for geospatial queries, which allows finding documents within a specific area, calculating distances between points.
3. Full-Test Search: Allows you to perform text search within your document


## Key Features of `MongoRepository`
1. CRUD operations
2. Query Derivation
3. Custom Queries with @Query

Define the entity class
```
@Document(collection = "user")
public class User {
    @Id
    private String id;
    private String firstname;
    private String lastname;
    private int age;

    // Constructs, getters, and setters
}
```

Create the UserRepository that extends MongoRepository for the User entity
```
public interface UserRepository extends MongoRepository<User, String> {
}
```
### CRUD Operations
1. Save
```
User saveUser(User user) {
    return UserRepository.save(uers);
}
```
2. Get
```
Optional<User> getUserById(String id) {
    return userRepository.findById(id);
}

List<User> getAllUsers() {
    return userRepository.findAll();
}
```
3. Delete
```
void deleteAllUsers() {
    userRepository.deleteAll();
}
```

### Query Derivation
Query derivation in MongoRepository allows you to define query methods in your repository interface based on method naming convention. Queries are generated from the method name, making it easy to perform queries other than the standared(inherited) ones without writing code.

How Query Derivation Works?<br>
Spring Data MongoDB parses the method names and creates appropriate queires based on the properties and operators specified in the method name.

```
public interface UserRepository extends MongoRepository<User, String> {
    // Find users by name
    List<User> findByName(String name);

    // Find users by age greater tha na specified value
    List<User> findByAgeGreaterThan(int age);

    // Find users by email
    List<User> findByEmail(String email);

    // Find users by name and age
    List<User> findByNameAndAge(String name, int age);

    // Find users by name containing a specific string (case insnsitive)
    List<User> findByNameContainingIgnoreCase(String name);
}
```

Method Naming Conventions:
1. Property Expressions: User the entity property names in the method name to indicate which fields to query on. Eg. `findByName(String name)` will generate a query based on the `name` property of the `User` entity.
2. Keywords: Spring Data MongoDB provides keywords to support common op-eations such as equity, comparison, and logical operations.
- - Equity:`findByName(String name)`
  - Comparison: `findByAgeGreaterThan(int age)`, `findByAgeLessThan(int age)`
  - Logical AND: `findByNameAndAge(String name, int age)`
  - Logical OR: `findByNameOrEmail(String name, String email)`
  - Like/Contains: `findByNameContaining(String name)`, `findByNameStartingWith(String prefix)`, `findByNameEndingWith(String suffix)`

### Custom Queries with @Query
Custom queries using the `@Query` annotation in Spring Data MongoDB allow you to define complex MongoDB queries directly within your repository interface. This approach provides a flexible way to write custom queries without needing to implement custom repository methods.

Define custome query methods in your repository using `@Query` annotation.
```
public interface UserRepository extends MongoRepository<User, String> {

    // Custom query method to find users by name
    @Query("{ 'name' : ?0 }")
    List<User> findUsersByName(String name);

    // Custom query method to find users older than a certain age
    @Query("{ 'age' : { '$gt' : ?0 } }")
    List<User> findUsersOlderThan(int age);

    // Custom query method to find users by email
    @Query("{ 'email' : ?0 }")
    List<User> findUsersByEmail(String email);

    // Custom query method to find users whose name contains a specific string (case insensitive)
    @Query("{ 'name' :  '$regex' : ?0, '$options' : '1' } }")
    List<User> findUsersByNameRegex(String regex);
}
```
## Custom Repository
Since Spring Data MongoDB does not support defining update queries directly using the `@Query` annotation in the repository interface. We can implement the update operation using `MongoTemplate` in a custom repository and have UserRepository extend the interface.

Step 1. Create Custom Repository Interface
```
public interface CustomUserRepository {
    User updateUser(String id, User userDetails);
}
```

Step 2: Implement the Custom Repository
```
@Repository
public class CustomUserRepositoryImpl implements CustomUserRepository {

    @Autowired
    private MongoTemplate mongoTemplate;

    @Override
    public User updateUser(String id, User userDetails) {
        Query query = new Query(Criteria.where("id).is(id));
        Update update = new Update()
                .set("name", userDetails.getName())
                .set("email", userDetails.getEmail())
                .set("age", userDetails.getAge());

        return mongoTemplate.findAndModify(query, update, FindAndModifyOptions.options().returnNew(true), User.class);
    }
}
```

Step 3: Extend the User Repository
```
public interface UserRepository extends MongoRepository<User, String>, CustomUserRepository {
    ...
}
```

Step 4. Use the Custom Update Method in the Service
```
@Service 
public class UserService {

    @Autowire
    private UserRepository userRepository;

    // Update User
    public User updateUser(String id, User userDetails) {
        return userRepository.updateUser(id, userDetails);
    }
}
```

## Pagination
We will implement a paginated custom query method using the `#Query` annotation and the `Pageable` interface.

Step 1: In the Repository Interface add Custom Query for Pagination
```
public interface UserRepository extends MongoRepository<User, String> {
    
    @Query("{}")
    Page<User> findAll(Pageable pageable);
}
```
The `{}` query matches all documents. This custom query enables pagination for fetching all users.

Step 2: Service Layer
```
@Service
public class UserService

    @Autowired
    private UserRepository userRepository;

    public Page<User> getAll(int page, int size) {
        Pageable pageable = PageRequest.of(page, size);
        return userRepository.findAll(pageable);
    }
}
```

## Projection
Spring Data MongoDB supports projections through interface-based projection or using the `Query` annotation.

Step 1. Define the projection method in the Repository.
```
public interface UserRepository extends MongoRepository<User, String> {

    @Query(value = "{}", fields = "{ 'name' : 1, 'email' : 1 }")
    List<User> findAll();
}
```
The `{}` query matches all documents, and the `fields` attribute specifies that only the `name` and `email` fields should be included in the result

Step 2. Service Layer
```
@Service
public class UserService {

    @Autowired
    private UserRepository userRepository;

    public List<User> getAll() {
        return userRepository.findAll();
    }
}
```


## Setup
MongoTemplate is from Spring Data Mongo not from Driver

first need to import the dependency in pom.xml
```
<dependency>
    <groupId>org.springframework.boot</groupId>
    <artifactId>spring-boot-starter-data-mongodb</artifactId>
</dependency>
```




MongoClient, MongoTemplate, MongoRepository

||MongoClient|MongoTemplate|MongoRepository|
|--|--|--|--|
|From|Java Mongo Driver|Spring Data Mongo|Spring Data Mongo|
|how to use it|Create MongoClient Object|Create Bean of MongoTemplate Ex: Java Config, Auto Configuration|Interface repository and extends MongoRepository|
|Result Object|Database Object Ex: BasicDbObject, Document|Entity class and domain objects both|Entity class Mandatory Ex. Book.class, Employee.class|
|Mapping|Need Mapping Explicitly|No Need of mapping with Entity class, with domain objects need|No need of Mapping reposity does that automatic|
|Projection|Possible through proper query|Through Built in Method chain|Possible with JSON Query|
|Aggregation|Possible through proper query|Through build in method chain|Not Possible|
|Code Size|Lots of code|Comparatively less Code|Almost no code|

# spring.jackson.default-property-inclusion=non-null
It is a property in Spring Boot application that is used to configure how Jackson library handles the serialization of object properties. Specifically it controls in inclusion criteria for properties when converting Java objects to JSON.

When you set `spring.jackson.default-property-inclusion=non-null`, you are instructing Jackson to include only those properties when serializing to JSON that has non-null values.

eg.
```
{
    "id": "1",
    "firstname": "John",
    "lastname": "Doe",
    "age": null
}
```

when you set `spring.jackson.default-property-inclusion=non-null`
```
{
    "id": "1",
    "firstname": "John",
    "lastname": "Doe",
}
```